In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## First Optimization Attempt
### Focus on dropping features for Model improvement

In [2]:
# Still drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],1)
application_df.head()

/var/folders/s9/jd3f2tsn0cgctyvrrylr_kdw0000gn/T/ipykernel_79722/2258305034.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(['EIN','NAME'],1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Review Features
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
application_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [5]:
application_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [6]:
# Features to remove from analysis
"""
Reasons for removing from analysis:
STATUS: 34294 to 5 in value_counts, even if informative, model will not learn association
SPECIAL_CONSIDERATIONS: 34272 to 27 in value_counts, even if informative, model will not learn association
"""
remove_features = ['STATUS','SPECIAL_CONSIDERATIONS']
application_df = application_df.drop(remove_features,1)
application_df.head()

/var/folders/s9/jd3f2tsn0cgctyvrrylr_kdw0000gn/T/ipykernel_79722/1166984781.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(remove_features,1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [7]:
# Determine which values to replace if counts are less than 1000
replace_application = [x for x in application_df['APPLICATION_TYPE'].unique() if application_df['APPLICATION_TYPE'].value_counts()[x] < 1000]

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [8]:
replace_class = [x for x in application_df['CLASSIFICATION'].unique() if application_df['CLASSIFICATION'].value_counts()[x] < 2000]

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [9]:
replace_inc = [x for x in application_df['INCOME_AMT'].unique() if application_df['INCOME_AMT'].value_counts()[x] < 20000]

# Replace in dataframe
for inc in replace_inc:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(inc,"1")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0    24388
1     9911
Name: INCOME_AMT, dtype: int64

In [10]:
# Generate our categorical variable lists
application_cat = [cat for cat in application_df.columns if application_df[cat].dtypes == 'object']
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat].values))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [12]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)
application_df.head()

/var/folders/s9/jd3f2tsn0cgctyvrrylr_kdw0000gn/T/ipykernel_79722/1085090237.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1
0,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [13]:
# Split our preprocessed data into our features and target arrays
target = 'IS_SUCCESSFUL'
y = application_df[target]
X = application_df.drop([target],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

/var/folders/s9/jd3f2tsn0cgctyvrrylr_kdw0000gn/T/ipykernel_79722/1815030161.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop([target],1)


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Train, test, evaluate model with altered data
#### Model Architecture to remain the same. 

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0,])
hidden_nodes_layer1 = 19
hidden_nodes_layer2 = 11
hidden_nodes_layer3 = 7
hidden_nodes_layer4 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1,
                            input_dim = number_input_features,
                            activation = 'relu')
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2,
                          activation='relu')
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3,
                          activation='relu')
)

# Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4,
                          activation='relu')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1,
                          activation='sigmoid')
)

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 19)                551       
                                                                 
 dense_1 (Dense)             (None, 11)                220       
                                                                 
 dense_2 (Dense)             (None, 7)                 84        
                                                                 
 dense_3 (Dense)             (None, 3)                 24        
                                                                 
 dense_4 (Dense)             (None, 1)                 4         
                                                                 
Total params: 883
Trainable params: 883
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [17]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq='epoch'
)

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50


2023-03-16 08:46:26.258695: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


745/804 [==========================>...] - ETA: 0s - loss: 0.6270 - accuracy: 0.6919
Epoch 1: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 1s 620us/step - loss: 0.6243 - accuracy: 0.6936
Epoch 2/50
771/804 [===========================>..] - ETA: 0s - loss: 0.5805 - accuracy: 0.7174
Epoch 2: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 0s 593us/step - loss: 0.5803 - accuracy: 0.7180
Epoch 3/50
750/804 [==========================>...] - ETA: 0s - loss: 0.5706 - accuracy: 0.7205
Epoch 3: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 0s 544us/step - loss: 0.5712 - accuracy: 0.7197
Epoch 4/50
714/804 [=========================>....] - ETA: 0s - loss: 0.5669 - accuracy: 0.7225
Epoch 4: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 0s 494us/step - loss: 0.5672 - accuracy: 0.7221
Epoch 5/50
722/804 [=========================>....]

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5620 - accuracy: 0.7269 - 169ms/epoch - 630us/step
Loss: 0.5619961023330688, Accuracy: 0.7268804907798767


## Second Attempt
### Focus on better binning of features

In [20]:
# Get data again into new DataFrame
application_df2 = pd.read_csv("Resources/charity_data.csv")
application_df2.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [21]:
# Perform above transformations

# Still drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df2 = application_df2.drop(['EIN','NAME'],1)
application_df2.head()

# Drop columns
remove_features = ['STATUS','SPECIAL_CONSIDERATIONS']
application_df2 = application_df2.drop(remove_features,1)
application_df2.head()

# Replace sparse values in APPLICATION_TYPE
replace_application = [x for x in application_df2['APPLICATION_TYPE'].unique() if application_df2['APPLICATION_TYPE'].value_counts()[x] < 1000]

# Replace in dataframe
for app in replace_application:
    application_df2.APPLICATION_TYPE = application_df2.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df2.APPLICATION_TYPE.value_counts()

# Replace sparse values in CLASSIFICATION
replace_class = [x for x in application_df2['CLASSIFICATION'].unique() if application_df2['CLASSIFICATION'].value_counts()[x] < 2000]

# Replace in dataframe
for cls in replace_class:
    application_df2.CLASSIFICATION = application_df2.CLASSIFICATION.replace(cls,"Other")




/var/folders/s9/jd3f2tsn0cgctyvrrylr_kdw0000gn/T/ipykernel_79722/3843103021.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df2 = application_df2.drop(['EIN','NAME'],1)
/var/folders/s9/jd3f2tsn0cgctyvrrylr_kdw0000gn/T/ipykernel_79722/3843103021.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df2 = application_df2.drop(remove_features,1)


In [22]:
# Bin INCOME_AMT to 0 or some amount of income.
replace_inc = [x for x in application_df2['INCOME_AMT'].unique() if application_df2['INCOME_AMT'].value_counts()[x] < 20000]

# Replace in dataframe
for inc in replace_inc:
    application_df2.INCOME_AMT = application_df2.INCOME_AMT.replace(inc,"1")
    
# Check to make sure binning was successful
application_df2.INCOME_AMT.value_counts()

0    24388
1     9911
Name: INCOME_AMT, dtype: int64

In [23]:
# Get ont hot vector of categorical variables
application_cat = [cat for cat in application_df2.columns if application_df2[cat].dtypes == 'object']
application_cat

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df2[application_cat].values))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

# Merge one-hot encoded features and drop the originals
application_df2 = application_df2.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)
application_df2.head()

/var/folders/s9/jd3f2tsn0cgctyvrrylr_kdw0000gn/T/ipykernel_79722/659922860.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df2 = application_df2.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1
0,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [24]:
# Split our preprocessed data into our features and target arrays
target = 'IS_SUCCESSFUL'
y = application_df2[target]
X = application_df2.drop([target],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

/var/folders/s9/jd3f2tsn0cgctyvrrylr_kdw0000gn/T/ipykernel_79722/4039037923.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df2.drop([target],1)


In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0,])
hidden_nodes_layer1 = 7
hidden_nodes_layer2 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1,
                            input_dim = number_input_features,
                            activation = 'relu')
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2,
                          activation='relu')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1,
                          activation='sigmoid')
)

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 7)                 203       
                                                                 
 dense_6 (Dense)             (None, 3)                 24        
                                                                 
 dense_7 (Dense)             (None, 1)                 4         
                                                                 
Total params: 231
Trainable params: 231
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [28]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq='epoch'
)

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
714/804 [=========================>....] - ETA: 0s - loss: 0.6146 - accuracy: 0.6883
Epoch 1: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 1s 495us/step - loss: 0.6110 - accuracy: 0.6912
Epoch 2/50
744/804 [==========================>...] - ETA: 0s - loss: 0.5849 - accuracy: 0.7099
Epoch 2: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 0s 492us/step - loss: 0.5837 - accuracy: 0.7101
Epoch 3/50
689/804 [========================>.....] - ETA: 0s - loss: 0.5795 - accuracy: 0.7149
Epoch 3: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 0s 504us/step - loss: 0.5787 - accuracy: 0.7145
Epoch 4/50
729/804 [==========================>...] - ETA: 0s - loss: 0.5768 - accuracy: 0.7170
Epoch 4: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 0s 491us/step - loss: 0.5762 - accuracy: 0.7182
Epoch 5/50
726/804 [====================

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5676 - accuracy: 0.7229 - 144ms/epoch - 539us/step
Loss: 0.5676294565200806, Accuracy: 0.7229154706001282


## Third attempt
### Changing model architecture only with changes to data from Attempt 2

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0,])
hidden_nodes_layer1 = 19
hidden_nodes_layer2 = 11
hidden_nodes_layer3 = 7
hidden_nodes_layer4 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1,
                            input_dim = number_input_features,
                            activation = 'relu')
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2,
                          activation='relu')
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3,
                          activation='relu')
)

# Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4,
                          activation='relu')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1,
                          activation='sigmoid')
)

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 19)                551       
                                                                 
 dense_9 (Dense)             (None, 11)                220       
                                                                 
 dense_10 (Dense)            (None, 7)                 84        
                                                                 
 dense_11 (Dense)            (None, 3)                 24        
                                                                 
 dense_12 (Dense)            (None, 1)                 4         
                                                                 
Total params: 883
Trainable params: 883
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [33]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
768/804 [===========================>..] - ETA: 0s - loss: 0.6059 - accuracy: 0.6933
Epoch 1: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 1s 464us/step - loss: 0.6042 - accuracy: 0.6945
Epoch 2/50
694/804 [========================>.....] - ETA: 0s - loss: 0.5717 - accuracy: 0.7196
Epoch 2: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 0s 442us/step - loss: 0.5709 - accuracy: 0.7203
Epoch 3/50
792/804 [============================>.] - ETA: 0s - loss: 0.5660 - accuracy: 0.7229
Epoch 3: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 0s 450us/step - loss: 0.5661 - accuracy: 0.7227
Epoch 4/50
692/804 [========================>.....] - ETA: 0s - loss: 0.5639 - accuracy: 0.7225
Epoch 4: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 0s 441us/step - loss: 0.5645 - accuracy: 0.7222
Epoch 5/50
801/804 [====================

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5630 - accuracy: 0.7257 - 156ms/epoch - 581us/step
Loss: 0.5629792809486389, Accuracy: 0.7257142663002014


In [35]:
nn.save("AlphabetSoupCharity_Optimization.h5")